In [1]:
%%px
from utils.creds import *
load_sf_creds()
username=os.environ["user"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="TEST_WH"
database=os.environ["dbname"]
schema="TPCH_SF10"
import sys
import bodo
rank=bodo.get_rank()
log = open(f"/tmp/process{rank}.log", "w")
sys.stderr = log

Starting 8 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

## Predicate Pushdown
Bodo does optimization in your code, when you are running a workload against snowflake.
In the below query, user is doing a naive Select * query into snowflake.
He is the filtering the dataframe based on Shipdate and dropping columns that are not needed.
In regular pandas, this would have caused unnecessary i/o and memory usage because all the data from snowflake, would have been brought it.
Bodo on the other hand will automatically detect these and push the filter and the needed columns into the query , hence improving i/o and memory utilization.
Run the cell below and check the query in snowflake history.

In [2]:
%%px
import bodo
import pandas as pd
@bodo.jit(cache=True)
def load_lineitem(schema):
    date = pd.Timestamp("1998-09-02")
    lineitem=pd.read_sql(f"select * from {schema}.LINEITEM", f"snowflake://{username}:{password}@{account}/{database}/PUBLIC?warehouse={warehouse}",)
    lineitem=lineitem[lineitem.l_shipdate <= date]
    lineitem=lineitem[["l_quantity", "l_shipdate"]]
    return lineitem
lineitem=load_lineitem(schema)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

In [3]:
%%px
lineitem.shape

Out[0:3]: (7392827, 2)

Out[7:3]: (7392826, 2)

Out[1:3]: (7392826, 2)

Out[6:3]: (7392826, 2)

Out[4:3]: (7392826, 2)

Out[2:3]: (7392826, 2)

Out[5:3]: (7392826, 2)

Out[3:3]: (7392826, 2)

In [4]:
%%px
#since we are on a small cluster, lets clear memory before running another query
del lineitem

## Superfast connector
Bodo also has an improved snowflake connector. If you notice the output of the above query, it is sharded across multiple cores of the cluster. Traditional etl tools, will do this sharding by doing an initial range query and then submitting multiple select queries for each of the range. Ex. if we are accessing a table employee, traditional etl tool with run the query with a count(*)
select count(*) from 'query'.
Once the count is established, the count will be divided by parallelism needed, lets say 8 in this case, 8 queries with different ranges will be pushed to snowflake.

This causes unnecessary load on the database and also is not truelly parallel. 
Bodo , will only submit one query to snowflake and the connector will receive sharded data from snowflake. 
Run the code below and check snowflake history, you should see only one query submitted.

In [5]:
%%px
import pandas as pd
import bodo
import time
@bodo.jit(cache=True)
def tpch_q01_filter(schema):
    t1 = time.time()
    lineitem=pd.read_sql(f"select * from {schema}.LINEITEM", f"snowflake://{username}:{password}@{account}/{database}/PUBLIC?warehouse={warehouse}",)
    date = pd.Timestamp("1998-09-02")
    sel = lineitem.l_shipdate <= date
    lineitem_filtered = lineitem[["l_quantity", "l_extendedprice", "l_discount", "l_tax", "l_returnflag", "l_linestatus",  "l_shipdate", "l_orderkey"]]
    lineitem_filtered = lineitem_filtered[sel]
    lineitem_filtered["avg_qty"] = lineitem_filtered.l_quantity
    lineitem_filtered["avg_price"] = lineitem_filtered.l_extendedprice
    lineitem_filtered["disc_price"] = lineitem_filtered.l_extendedprice * (1 - lineitem_filtered.l_discount)
    lineitem_filtered["charge"] = (
        lineitem_filtered.l_extendedprice * (1 - lineitem_filtered.l_discount) * (1 + lineitem_filtered.l_tax)
    )
    gb = lineitem_filtered.groupby(["l_returnflag", "l_linestatus"], as_index=False)[
        "l_quantity",
        "l_extendedprice",
        "disc_price",
        "charge",
        "avg_qty",
        "avg_price",
        "l_discount",
        "l_orderkey",
    ]
    total = gb.agg(
        {
            "l_quantity": "sum",
            "l_extendedprice": "sum",
            "disc_price": "sum",
            "charge": "sum",
            "avg_qty": "mean",
            "avg_price": "mean",
            "l_discount": "mean",
            "l_orderkey": "count",
        }
    )
    total = total.sort_values(["l_returnflag", "l_linestatus"])
    print(len(total))
    print("Q01 Execution time (s): ", time.time() - t1)
    return total

q1_result=tpch_q01_filter(schema)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] 4
Q01 Execution time (s):  17.53355710091637


In [ ]:
%%px
from utils.tpch_functions import *
lineitem=load_lineitem(schema)
orders=load_orders(schema)
customer=load_customer(schema)
nation=load_nation(schema)
region=load_region(schema)
supplier=load_supplier(schema)
q05(lineitem, orders, customer, nation, region, supplier)

In [4]:
%%px
lineitem.shape

Out[2:4]: (2464276, 16)

Out[0:4]: (2464276, 16)

Out[5:4]: (2464276, 16)

Out[12:4]: (2464275, 16)

Out[6:4]: (2464276, 16)

...

Out[19:4]: (2464275, 16)

Out[21:4]: (2464275, 16)

Out[22:4]: (2464275, 16)

Out[23:4]: (2464275, 16)

Out[16:4]: (2464275, 16)

In [6]:
%%px
q1_result

,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey


,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey
0,A,F,377518399.0,5.660657e+11,5.377591e+11,5.592767e+11,25.500975,38237.151009,0.050007,14804077
1,N,F,9851614.0,1.476744e+10,1.402881e+10,1.459049e+10,25.522448,38257.810660,0.049973,385998
3,N,O,743124873.0,1.114302e+12,1.058581e+12,1.100937e+12,25.498076,38233.902923,0.050001,29144351
2,R,F,377732830.0,5.664311e+11,5.381109e+11,5.596348e+11,25.508385,38251.219274,0.049997,14808183
